## Import Libraries

In [1]:
import captum
import numpy as np
import torch
import torch.nn as nn

from captum.attr import (
    GradientShap,
    DeepLift,
    DeepLiftShap,
    IntegratedGradients,
    LayerConductance,
    NeuronConductance,
    NoiseTunnel,
)

### Basic Neural Network Model

In [2]:
class BasicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(10, 10)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(10, 2)

    def forward(self, input):
        return self.lin2(self.relu(self.lin1(input)))

In [3]:
model = BasicModel()
model.eval()

BasicModel(
  (lin1): Linear(in_features=10, out_features=10, bias=True)
  (relu): ReLU()
  (lin2): Linear(in_features=10, out_features=2, bias=True)
)

## Defining The Input & Baseline Tensors

### Baseline Tensor : Tensor that holds the zero interpretability power. 

In [4]:
input = torch.rand(5, 10)
baseline = torch.zeros(5, 10)

## General Attribution

### Integrated Gradients

Assigning Attribution Score to each input feature with respect to the first target output. 

Positive Attribution scores implies higher contribution by the input for that particular prediction and Negative Attribution score implies opposite. 

Lower the convergence Delta (Approximation Error) , the better is the approximation. If the absolute value of convergence delta is large, we can try large number of integralapproximation steps. 


In [6]:
ig = IntegratedGradients(model)
attributions, delta = ig.attribute(input, baseline, target=0, return_convergence_delta=True)
print('IG Attributions:', attributions)
print('Convergence Delta:', delta)

IG Attributions: tensor([[-0.0004,  0.0497, -0.0143,  0.0190,  0.0003,  0.0917, -0.0160,  0.0481,
         -0.0048,  0.0249],
        [-0.0064,  0.0393, -0.0562,  0.0399, -0.0030,  0.0551, -0.0033,  0.0614,
         -0.0055,  0.0144],
        [-0.0102,  0.0297, -0.0645,  0.0562, -0.0097,  0.0337,  0.0005,  0.0600,
         -0.0623,  0.0082],
        [-0.0064,  0.0359, -0.0150,  0.0215, -0.0030,  0.0672,  0.0006,  0.0491,
         -0.0682,  0.0185],
        [-0.0044,  0.0556, -0.0940,  0.0269, -0.0140,  0.0643,  0.0085,  0.0461,
         -0.0139,  0.0135]])
Convergence Delta: tensor([-0.0009, -0.0008, -0.0010, -0.0008,  0.0002])


### Gradient Shap

1. It first chooses the random baseline from baseline distribution.
2. Then is adds the gaussian noise to each input example. (It adds it n_samples times)
3. Then it chooses a random point between each example-baseline pair and computes the gradients with respect to target class (in this case target=0)


In [7]:
gs = GradientShap(model)

# We define a distribution of baselines and draw `n_samples` from that
# distribution in order to estimate the expectations of gradients across all baselines
baseline_dist = torch.randn(5, 10) * 0.001
attributions, delta = gs.attribute(input, stdevs=0.09, n_samples=4, baselines=baseline_dist,
                                   target=0, return_convergence_delta=True)
print('GradientShap Attributions:', attributions)
print('Convergence Delta:', delta)

GradientShap Attributions: tensor([[ 0.0003,  0.0582, -0.0160,  0.0153, -0.0016,  0.0852,  0.0071,  0.0325,
          0.0016,  0.0085],
        [-0.0075,  0.0424, -0.0373,  0.0098, -0.0045,  0.0602, -0.0089,  0.0431,
         -0.0115,  0.0043],
        [-0.0093,  0.0276, -0.0683,  0.0664, -0.0023,  0.0303, -0.0008,  0.0652,
         -0.0544,  0.0104],
        [-0.0005,  0.0333, -0.0079,  0.0244,  0.0033,  0.0729, -0.0078,  0.0641,
         -0.0803,  0.0257],
        [-0.0045,  0.0672, -0.1061,  0.0371, -0.0109,  0.0805,  0.0095,  0.0511,
         -0.0127,  0.0215]])
Convergence Delta: tensor([ 0.0070, -0.0681,  0.0002,  0.0244, -0.0954,  0.0005,  0.0002, -0.1033,
         0.0319,  0.0298,  0.0322,  0.0322,  0.0005,  0.0296,  0.0436, -0.0019,
         0.0319,  0.0591,  0.0239,  0.0320])


## Neuron Conductance

1. It helps us to identify input features that are important for a particular neuron in a given layer.

2.  It decomposes the computation of integrated gradients via the chain rule by defining the importance of a neuron as path integral of the derivative of the output with respect to the neuron times the derivatives of the neuron with respect to the inputs of the model.

In [8]:
nc = NeuronConductance(model, model.lin1)
attributions = nc.attribute(input, neuron_index=1, target=0)
print('Neuron Attributions:', attributions)

Neuron Attributions: tensor([[ 1.7536e-03,  7.7569e-03, -3.2716e-03, -1.1427e-02, -5.7137e-05,
          9.0864e-03,  3.8092e-03, -5.0003e-03, -8.8837e-04, -1.9285e-02],
        [ 5.3203e-03,  5.6370e-03, -1.0833e-02, -1.4767e-02, -4.6135e-04,
          5.5448e-03,  4.6199e-03, -6.3303e-03, -1.0119e-03, -1.1207e-02],
        [ 5.2308e-03,  3.9608e-03, -1.1348e-02, -1.6334e-02, -5.2938e-04,
          3.3745e-03,  3.0249e-04, -6.2707e-03, -1.2015e-02, -5.9522e-03],
        [ 3.8003e-03,  4.8386e-03, -3.0111e-03, -8.3664e-03, -1.6314e-04,
          6.7571e-03,  2.1895e-03, -5.5597e-03, -1.3861e-02, -1.4895e-02],
        [ 2.4302e-03,  7.0696e-03, -1.8524e-02, -9.9496e-03, -4.5020e-04,
          6.8692e-03,  5.3953e-03, -6.2380e-03, -3.2197e-03, -1.2156e-02]])


## Layer Conductance

It shows the importance of neurons for a layer and given input. It is an extension of path integrated gradients for hidden layers and holds the completeness property as well.

In [9]:
lc = LayerConductance(model, model.lin1)
attributions, delta = lc.attribute(input, baselines=baseline, target=0, return_convergence_delta=True)
print('Layer Attributions:', attributions)
print('Convergence Delta:', delta)

Layer Attributions: tensor([[ 0.0808, -0.0179, -0.0667,  0.1048,  0.0449,  0.0332,  0.0000,  0.0331,
         -0.0142,  0.0000],
        [ 0.0489, -0.0239, -0.0530,  0.0537,  0.0669,  0.0328,  0.0000,  0.0324,
         -0.0219,  0.0000],
        [ 0.0583, -0.0403, -0.0768,  0.0709,  0.0667, -0.0375,  0.0000,  0.0003,
         -0.0006,  0.0000],
        [ 0.0836, -0.0288, -0.0681,  0.0772,  0.0325, -0.0311,  0.0000,  0.0319,
          0.0021,  0.0000],
        [ 0.0487, -0.0293, -0.0469,  0.0479,  0.0204,  0.0426,  0.0000,  0.0319,
         -0.0273,  0.0000]], grad_fn=<SumBackward1>)
Convergence Delta: tensor([-0.0012, -0.0008, -0.0018, -0.0017, -0.0004])
